In [1]:
import src.data_prep
import librosa
import librosa.display
import soundfile as sf
import IPython.display as ipd

import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import glob
import os

%matplotlib inline


In [2]:
ls

EDA.ipynb                        Untitled.ipynb
README.md                        data/
Speech_Detection_Pipeline.ipynb  src/


# Organizing paths to audio and text files

In [3]:
audio_paths, texts = src.data_prep.get_audio_and_text_data('/Users/joshbernd/Desktop/gal_notes/Capstone/Speech_Detection/data/LibriSpeech_train/train-clean-100/')        


text file:  0
text file:  50
text file:  100
text file:  150
text file:  200
text file:  250
text file:  300
text file:  350
text file:  400
text file:  450
text file:  500
text file:  550


Here is how the data looks:

In [4]:
audio_paths[111]#, texts[111]

'/Users/joshbernd/Desktop/gal_notes/Capstone/Speech_Detection/data/LibriSpeech_train/train-clean-100/8580/287364/8580-287364-0003.flac'

## Let's store our phonemes and text for safe keeping

with open("data/ipa.txt", "w") as f:
    for t in texts:
        f.write(str(t) +"\n")

with open("data/audio_path.txt", "w") as f:
    for a in audio_paths:
        f.write(str(a) + "\n")

In [6]:
## reading in saved texts
texts = []
with open("data/ipa.txt", "r") as f:
  for line in f:
    texts.append(line.strip())

In [7]:
audio_paths = []
with open('data/audio_path.txt', 'r') as f:
    for line in f:
        audio_paths.append(line.strip())

In [8]:
texts[0], audio_paths[0]

('hæd leɪd ˌbiˈfɔr hər ə pɛr əv ɔlˈtərnətɪvz naʊ əv kɔrs jʊr kəmˈplitli jʊr oʊn ˈmɪstrəs ənd ər ɛz fri ɛz ðə bərd ɔn ðə baʊ aɪ doʊnt min ju wər nɑt soʊ ˌbiˈfɔr bət jʊr æt ˈprɛzənt ɔn ə ˈdɪfərənt ˈfʊtɪŋ',
 '/Users/joshbernd/Desktop/gal_notes/Capstone/Speech_Detection/data/LibriSpeech_train/train-clean-100/1069/133709/1069-133709-0000.flac')

## Now that we have the paths to the .flac files, let's convert them to spectrograms

In [9]:
a = src.data_prep.spectrogram_from_file(audio_paths[0])

/usr/local/anaconda3/lib/python3.8/site-packages/librosa/core/audio.py:162: UserWarning: PySoundFile failed. Trying audioread instead.
  warnings.warn("PySoundFile failed. Trying audioread instead.")


In [11]:
len(a)

1508

In [17]:
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import layers

In [20]:
model = keras.Sequential()
model.add(layers.Embedding(input_dim=161, output_dim=46))

# The output of GRU will be a 3D tensor of shape (batch_size, timesteps, 256)
model.add(layers.GRU(256, return_sequences=True))

# The output of SimpleRNN will be a 2D tensor of shape (batch_size, 128)
model.add(layers.SimpleRNN(128))

model.add(layers.Dense(10))

model.summary()

Model: "sequential_2"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding (Embedding)        (None, None, 46)          7406      
_________________________________________________________________
gru (GRU)                    (None, None, 256)         233472    
_________________________________________________________________
simple_rnn (SimpleRNN)       (None, 128)               49280     
_________________________________________________________________
dense_2 (Dense)              (None, 10)                1290      
Total params: 291,448
Trainable params: 291,448
Non-trainable params: 0
_________________________________________________________________


-------

## Unrelated - nn practice with MNIST

In [24]:
batch_size = 64
# Each MNIST image batch is a tensor of shape (batch_size, 28, 28).
# Each input sequence will be of size (28, 28) (height is treated like time).
input_dim = 28

units = 64
output_size = 10  # labels are from 0 to 9

# Build the RNN model
def build_model(allow_cudnn_kernel=True):
    # CuDNN is only available at the layer level, and not at the cell level.
    # This means `LSTM(units)` will use the CuDNN kernel,
    # while RNN(LSTMCell(units)) will run on non-CuDNN kernel.
    if allow_cudnn_kernel:
        # The LSTM layer with default options uses CuDNN.
        lstm_layer = keras.layers.LSTM(units, input_shape=(None, input_dim))
    else:
        # Wrapping a LSTMCell in a RNN layer will not use CuDNN.
        lstm_layer = keras.layers.RNN(
            keras.layers.LSTMCell(units), input_shape=(None, input_dim)
        )
    model = keras.models.Sequential(
        [
            lstm_layer,
            keras.layers.BatchNormalization(),
            keras.layers.Dense(output_size),
        ]
    )
    return model

In [25]:
mnist = keras.datasets.mnist

(x_train, y_train), (x_test, y_test) = mnist.load_data()
x_train, x_test = x_train / 255.0, x_test / 255.0
sample, sample_label = x_train[0], y_train[0]

In [28]:
model = build_model(allow_cudnn_kernel=False)

model.compile(
    loss=keras.losses.SparseCategoricalCrossentropy(from_logits=True),
    optimizer="sgd",
    metrics=["accuracy"],
)


model.fit(
    x_train, y_train, validation_data=(x_test, y_test), batch_size=batch_size, epochs=5
)

Epoch 1/5
938/938 [==============================] - 13s 14ms/step - loss: 0.9434 - accuracy: 0.7010 - val_loss: 0.5595 - val_accuracy: 0.8251
Epoch 2/5
938/938 [==============================] - 11s 12ms/step - loss: 0.3947 - accuracy: 0.8790 - val_loss: 0.3552 - val_accuracy: 0.8902
Epoch 3/5
938/938 [==============================] - 9s 10ms/step - loss: 0.2610 - accuracy: 0.9215 - val_loss: 0.2289 - val_accuracy: 0.9309
Epoch 4/5
938/938 [==============================] - 9s 10ms/step - loss: 0.2015 - accuracy: 0.9387 - val_loss: 0.2414 - val_accuracy: 0.9200
Epoch 5/5
938/938 [==============================] - 9s 10ms/step - loss: 0.1686 - accuracy: 0.9490 - val_loss: 0.1683 - val_accuracy: 0.9449


In [12]:
from keras.models import Model
from keras.layers import Input, GRU, Activation

In [15]:
def simple_rnn_model(input_dim, output_dim=46):
    """ Build a recurrent network for speech 
    """
    # Main acoustic input
    input_data = Input(name='the_input', shape=(None, input_dim))
    # Add recurrent layer
    simp_rnn = GRU(output_dim, return_sequences=True, 
                 implementation=2, name='rnn')(input_data)
    # Add softmax activation layer
    y_pred = Activation('softmax', name='softmax')(simp_rnn)
    # Specify the model
    model = Model(inputs=input_data, outputs=y_pred)
    model.output_length = lambda x: x
    print(model.summary())
    return model

In [16]:
model_0 = simple_rnn_model(input_dim=161)

Model: "functional_3"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
the_input (InputLayer)       [(None, None, 161)]       0         
_________________________________________________________________
rnn (GRU)                    (None, None, 46)          28842     
_________________________________________________________________
softmax (Activation)         (None, None, 46)          0         
Total params: 28,842
Trainable params: 28,842
Non-trainable params: 0
_________________________________________________________________
None


In [ ]:
train_model(input_to_softmax=model_0, 
            pickle_path='model_0.pickle', 
            save_model_path='model_0.h5',
            spectrogram=True)